In [ ]:
#실패한 리스트 링크 feature수집

In [ ]:
#패키지 로드 및 크롬 드라이브 연결

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import operator
import numpy as np
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome('C:/Users/tlath/Desktop/chromedriver')

In [ ]:

fail_link_list = list()


# kickstater 로그인 페이지 사이트로 들어가기
driver.get('https://www.kickstarter.com/login?ref=nav')

#find_elemnet_by를 이용하여 아이디, 비밀번호 입력하기
driver.find_element_by_id('user_session_email').send_keys('ID')
driver.find_element_by_id('user_session_password').send_keys('PASSWORD')

#아이디, 비밀번호 입력 후, 로그인 버튼의 태그값을 찾아 로그인 하기
xpath = """//*[@id="new_user_session"]/fieldset/ol/li[3]/input"""
driver.find_element_by_xpath(xpath).click()

#프로젝트를 saved한 페이지 링크 URL
url="https://www.kickstarter.com/profile/saved_projects?ref=user_menu"

driver.get(url)
time.sleep(0.2)
body = driver.find_element_by_tag_name("body")
num = 100

# 무한 로드 버튼을 찾아서 끝까지 클릭하기!
while num:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)
    num -= 1
    try:
        driver.find_element_by_xpath("""//*[@id="react-container"]/div[2]/div[3]/div/button""").click()
    except:
        None
        
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 개발자도구 > COPT > SELECT 부분의 값을 가져와서 프로젝트 타이틀 수집하기
a_tags = soup.find_all("a", {'class':"block img-placeholder w100p"}) 

for line in a_tags :
    title = line["href"]
    print(title)
    fail_link_list.append(title)

In [ ]:
# 수집되는 Featrue들을 넣기 위해 각 Feature별로 리스트 생성
money_list=[]
pledged_money_list=[]
creator_list=[]
backer_list=[]
created_list=[]
city_list=[]
category_list=[]
support_bankroll_list=[]
support_backers_list = []
level_num_list=[]
support_max_money_list = []
support_min_money_list = []
support_max_backers_list = []
support_min_backers_list = []


In [ ]:
for i in range(len(fail_link_list)):
    url = fail_link_list[i]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    print('링크',url)
    time.sleep(8)
      
    #실제로모인금액
    money = soup.select('span.soft-black')[1].text     
    money= re.findall('\d+',money)
    money= "".join(money)
    money_list.append(money)
    print('실제로 모인 금액',money)
    time.sleep(0.2)
    
    #목표금액
    pledged_money = soup.find('span',{'class':'inline-block hide-sm'}).text
    pledged_money= re.findall('\d+',pledged_money)
    pledged_money= "".join(pledged_money)
    pledged_money_list.append(pledged_money)
    print('목표금액',pledged_money)
    time.sleep(0.2)
    
    #제작자이름
    creator = soup.select('div.text-left')[1].text
    creator_list.append(creator)
    print('제작자이름',creator)
    
    #후원자수
    time.sleep(0.2)
    backers = soup.select(' div > div > div > div > div.flex.flex-column-lg.mb4.mb5-sm > div.ml5.ml0-lg.mb4-lg > div > span')[0].text
    backer_list.append(backers)    
    print('후원자수',backers)
    time.sleep(0.2)
    
    #제작자의프로젝트개설횟수
    created= soup.select('div.text-left')[2].text
    final_created= re.findall('\d+',created)
    for i in range(len(final_created)):
        final_created[i]=int(final_created[i])
    if len(final_created)==1:
        created_num=0
    else:
        created_num=final_created[0]
    created_list.append(created_num)
    print('제작개설횟수',created_num)   
    #time.sleep(0.2)
    
    #제작자지역
    city = soup.select('span.ml1')[1].text
    city_list.append(city)
    print('제작자지역',city)
    time.sleep(0.2)
    
    #프로젝트장르
    category = soup.select('span.ml1')[0].text
    category_list.append(category)
    print('프로젝트 장르',category)
    time.sleep(0.2)

    #서포트금액
    support_money = soup.select('div.NS_projects__content > section.js-project-content.js-project-description-content.project-content > div > div > div > div.col.col-4.max-w62.sticky-rewards.z10 > div > div.mobile-hide > div > ol > li > div.pledge__info > h2 > span.money')
    support_bankroll=[]
    for line in support_money:
        line=line.get_text()         
        line= re.findall('\d+',line)
        line = "".join(line)
        support_bankroll.append(int(line)) 

    support_bankroll_list.append(support_bankroll)
    print('서포트금액', support_bankroll)
    time.sleep(0.2)
    
    #서포트후원자수
    support_backer = soup.select('div.NS_projects__content > section.js-project-content.js-project-description-content.project-content > div > div > div > div.col.col-4.max-w62.sticky-rewards.z10 > div > div.mobile-hide > div > ol > li > div.pledge__info > div.pledge__backer-stats > span')
    support_backers = []
    for line in support_backer:
        line = line.get_text() 
        if line == 'Includes:':
            continue
        line = line.replace('\n','')
        if line == 'Limited':
            continue
        if line == 'Reward no longer available':
            continue
        line = line.replace(' backers','')
        line = line.replace(' backer','')
        support_backers.append(line)
    support_backers_list.append(support_backers)
    print("서포트 금액별 후원자수",support_backers)   
    time.sleep(0.2)
    
    #단계개수
    level_num=len(support_bankroll)
    level_num_list.append(level_num)
    print('단계',level_num)
    time.sleep(0.2)
    
    
    #서포트금액 최소/최대
    support_max_idx = 0
    support_min_idx = 0
    max = support_bankroll[0]
    min = support_bankroll[0]
    
    for i in range(len(support_bankroll)):  
        if max < support_bankroll[i]:
            max = support_bankroll[i]
            support_max_idx = i
        if min > support_bankroll[i]:
            min = support_bankroll[i]
            support_min_idx = i
    print("최대금액",max)
    print("최소금액",min)
    support_max_money_list.append(max)
    support_min_money_list.append(min)
    time.sleep(0.2)
    
    #서포트최소/최대후원자수
    a_ar = np.array(support_bankroll)
    max_money_index = np.argmax(support_bankroll)
    min_money_index = np.argmin(support_bankroll)
    max_money_backers = support_backers[max_money_index]
    min_money_backers = support_backers[min_money_index]
    support_max_backers_list.append(max_money_backers)
    support_min_backers_list.append(min_money_backers)
    print("최대 금액 후원자수",max_money_backers)
    print("최소 금액 후원자수",min_money_backers)
    print('\n')
    time.sleep(0.2)

In [ ]:
fail_list = []
result_fail = []

for money_list,pledged_money_list,creator_list,backer_list,created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list in zip(money_list,pledged_money_list,creator_list,backer_list,created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list):       
    fail_list = [money_list,pledged_money_list,creator_list,backer_list,created_list,city_list,category_list,support_bankroll_list,support_max_money_list,support_min_money_list,level_num_list,support_backers_list,support_max_backers_list,support_min_backers_list]
    result_fail.append(fail_list)

In [ ]:
import pandas as pd
df_fail = pd.DataFrame(result_fail, columns = ['money','pledged_money','creator','backers','Number of projects opened','Producer area','genre','Support amount list','The highest amount of support','The minimum amount of support','Total number of support steps','List of supporters','Maximum amount of sponsors','Minimum amount of sponsors'])
df_fail.to_csv('fail_link_feature.csv',encoding = 'utf-8-sig')